In [4]:
import pandas as pd
import numpy as np
from dateutil import parser

# News Data Processing

In [12]:
def convert_to_month_day_year(datetime_str):
    parsed_datetime = parser.parse(str(datetime_str))
    formatted_datetime = parsed_datetime.strftime('%B %d, %Y')
    return formatted_datetime

In [3]:
cnbc_df = pd.read_csv('cnbc_headlines.csv', usecols=['Headlines', 'Time'])
cnbc_df.dropna(inplace=True)
print(cnbc_df.shape)

cnbc_df['Time'] = cnbc_df['Time'].apply(convert_to_month_day_year)
cnbc_df.head(2)

(2800, 2)


/opt/homebrew/Caskroom/miniforge/base/envs/jupyter/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Headlines,Time
0,Jim Cramer: A better way to invest in the Covi...,"July 17, 2020"
1,Cramer's lightning round: I would own Teradyne,"July 17, 2020"


In [4]:
guardian_df = pd.read_csv('guardian_headlines.csv')
guardian_df.dropna(inplace=True)
print(guardian_df.shape)

guardian_df['Time'] = guardian_df['Time'].apply(convert_to_month_day_year)
guardian_df.head(2)

(17800, 2)


,Time,Headlines
0,"July 18, 2020",Johnson is asking Santa for a Christmas recovery
1,"July 18, 2020",‘I now fear the worst’: four grim tales of wor...


In [5]:
reuters_df = pd.read_csv('reuters_headlines.csv', usecols=['Headlines', 'Time'])
reuters_df.dropna(inplace=True)
print(reuters_df.shape)

reuters_df['Time'] = reuters_df['Time'].apply(convert_to_month_day_year)
reuters_df.head(2)

(32770, 2)


,Headlines,Time
0,TikTok considers London and other locations fo...,"July 18, 2020"
1,Disney cuts ad spending on Facebook amid growi...,"July 18, 2020"


In [6]:
import os
import json 


folder_paths = ['./2018_1', './2018_2', './2018_3', './2018_4', './2018_5'] 

data = []

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)
            data.append(json_data)

json_news_df = pd.json_normalize(data)
json_news_df = json_news_df[['title', 'thread.published']]

json_news_df.dropna(inplace=True)
print(json_news_df.shape)

(306242, 2)


In [8]:
json_news_df['thread.published'] = json_news_df['thread.published'].apply(convert_to_month_day_year)
json_news_df.rename(columns={'title': 'Headlines', 'thread.published': 'Time'}, inplace=True)
json_news_df.head(2)

,Headlines,Time
0,EMERGING MARKETS-Mexican peso seesaws over dol...,"January 26, 2018"
1,"Migrants must visit Nazi concentration camps, ...","January 10, 2018"


In [18]:
# Combine all the dfs
news_df = pd.concat([json_news_df, reuters_df, guardian_df, cnbc_df], axis=0)
print(news_df.shape)
news_df.head(2)

(359612, 2)


,Headlines,Time
0,EMERGING MARKETS-Mexican peso seesaws over dol...,"January 26, 2018"
1,"Migrants must visit Nazi concentration camps, ...","January 10, 2018"


In [19]:
# Remove the duplicated rows
before_rows = news_df.shape[0]
news_df.drop_duplicates(inplace=True)

print(f"Removed {before_rows - news_df.shape[0]} rows as duplicates. There are now {news_df.shape[0]} rows.")

Removed 39074 rows as duplicates. There are now 320538 rows.


In [20]:
news_df.to_csv("./news.csv", index=False)

# Macro-Econ Data

In [5]:
econ_df = pd.read_csv("DATA.csv")
econ_df.head(3)

,DATE,UNRATE(%),CONSUMER CONF INDEX,PPI-CONST MAT.,CPIALLITEMS,INFLATION(%),MORTGAGE INT. MONTHLY AVG(%),MED HOUSEHOLD INCOME,CORP. BOND YIELD(%),MONTHLY HOME SUPPLY,% SHARE OF WORKING POPULATION,GDP PER CAPITA,QUARTERLY REAL GDP,QUARTERLY GDP GROWTH RATE (%),CSUSHPISA
0,01-05-2022,3.6,106.4,352.857,123.322800,8.581511,5.2300,NaN,4.13,8.4,NaN,74737,19699.465,-0.144227,120.724
1,01-04-2022,3.6,107.3,343.730,121.978170,8.258629,4.9825,NaN,3.76,8.4,NaN,74737,19699.465,-0.144227,121.813
2,01-03-2022,3.6,107.2,345.852,121.301004,8.542456,4.1720,NaN,3.43,7.0,NaN,73289,19727.918,-0.395692,122.888


In [13]:
imp_econ_df = econ_df[['DATE', 'UNRATE(%)', 'INFLATION(%)', 'GDP PER CAPITA', 'QUARTERLY REAL GDP', 'QUARTERLY GDP GROWTH RATE (%)']]

imp_econ_df.rename(columns={'DATE': 'date', 
                            'UNRATE(%)': 'unemployment rate (%)', 
                            'INFLATION(%)': 'inflation rate (%)',
                            'GDP PER CAPITA': 'gdp per capita',
                            'QUARTERLY REAL GDP': 'quarterly real GDP',
                            'QUARTERLY GDP GROWTH RATE (%)': 'quarterly gdp growth rate (%)'}, inplace=True)

imp_econ_df['date'] = imp_econ_df['date'].apply(convert_to_month_day_year)
imp_econ_df.head(3)

/var/folders/qg/g7gs2bjn2llf1p8c229qrymw0000gn/T/ipykernel_5057/1898289180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imp_econ_df.rename(columns={'DATE': 'date',
/var/folders/qg/g7gs2bjn2llf1p8c229qrymw0000gn/T/ipykernel_5057/1898289180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imp_econ_df['date'] = imp_econ_df['date'].apply(convert_to_month_day_year)


,date,unemployment rate (%),inflation rate (%),gdp per capita,quarterly real GDP,quarterly gdp growth rate (%)
0,"January 05, 2022",3.6,8.581511,74737,19699.465,-0.144227
1,"January 04, 2022",3.6,8.258629,74737,19699.465,-0.144227
2,"January 03, 2022",3.6,8.542456,73289,19727.918,-0.395692


In [10]:
imp_econ_df.to_csv("econ_data.csv", index=False)